<a href="https://colab.research.google.com/github/CrasCris/MachineLearning/blob/main/Predicci%C3%B3nTRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
#Liberias
import numpy as np
import time as tm
import datetime as dt
import tensorflow as tf
import pandas as pd
# Data preparation
from yahoo_fin import stock_info as yf
from sklearn.preprocessing import MinMaxScaler
from collections import deque
from datetime import timedelta

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt

In [ ]:
#Las que toca Instalar -pip install yahoo_fin-

In [70]:
#Configuraciones

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

# Stock ticker, GOOGL
STOCK = 'GOOGL'

# Current date
date_now = tm.strftime('%d/%m/%Y')
date_3_years_back = (dt.date.today() - dt.timedelta(days=1104)).strftime('%d/%m/%Y')

In [71]:
# Cargando datos recordar subir el archivo al colab y copiar la ruta para subir !OJO PUEDE NO SER LA MISMA RUTA QUE SE MUESTRA AQUI¡
datos = pd.read_csv("/content/Tasa_de_Cambio_Representativa_del__Mercado_-Historico_20231013.csv")
datos

,VALOR,UNIDAD,VIGENCIADESDE,VIGENCIAHASTA
0,4761.64,COP,22/12/2022,22/12/2022
1,4781.28,COP,20/12/2022,20/12/2022
2,4802.48,COP,17/12/2022,19/12/2022
3,4836.24,COP,13/12/2022,13/12/2022
4,4815.99,COP,10/12/2022,12/12/2022
...,...,...,...,...
7654,4359.40,COP,06/10/2023,06/10/2023
7655,4386.66,COP,07/10/2023,10/10/2023
7656,4231.97,COP,11/10/2023,11/10/2023
7657,4212.50,COP,12/10/2023,12/10/2023


In [72]:
# Asegúrate de que 'Fecha' sea un objeto datetime
datos['VIGENCIAHASTA'] = pd.to_datetime(datos['VIGENCIAHASTA'],format='%d/%m/%Y')

datos['VIGENCIADESDE'] = pd.to_datetime(datos['VIGENCIADESDE'],format='%d/%m/%Y')
# Ordena el DataFrame por la columna 'Fecha' de manera descendente
datosOrd=datos.sort_values(by='VIGENCIAHASTA', ascending=False)

In [74]:
#Agregando los valores faltantes de la serie
date_ranges = []
for index, row in datosOrd.iterrows():
    trm = row['VALOR']
    fecha_desde = row['VIGENCIADESDE']
    fecha_hasta = row['VIGENCIAHASTA']

    date_range = pd.date_range(start=fecha_desde, end=fecha_hasta, freq='D')

    for fecha in date_range:
        date_ranges.append({'VALOR': trm, 'Fecha': fecha})

new_df = pd.DataFrame(date_ranges)

Serie = new_df.sort_values(by='Fecha', ascending=True)



In [ ]:
#Grafica de la serie
plt.figure(figsize=(12, 6))
plt.plot(Serie['Fecha'], Serie['VALOR'], marker='o', linestyle='-', linewidth=1, markersize=4)
plt.title('Valores del TRM a lo largo del tiempo')
plt.xlabel('Fecha')
plt.ylabel('VALOR')
plt.grid(True)
plt.xticks(rotation=45)

In [ ]:
#Grafica de la serie usando el estilo GGPLOT
plt.style.use(style='ggplot')
plt.figure(figsize=(16,10))
plt.plot(Serie['Fecha'], Serie['VALOR'], marker='o', linestyle='-', linewidth=1, markersize=4)
plt.xlabel("FECHA")
plt.ylabel("price")
plt.show()

In [75]:
#Escalando la data
scaler = MinMaxScaler()
Serie['VALOR'] = scaler.fit_transform(np.expand_dims(Serie['VALOR'] .values, axis=1))

In [101]:
#Modelo de maching learning
def PrepareData(days):
  df = Serie.copy()
  df['future'] = df['VALOR'].shift(-days)
  last_sequence = np.array(df[['VALOR']].tail(days))
  df.dropna(inplace=True)
  sequence_data = []
  sequences = deque(maxlen=N_STEPS)

  for entry, target in zip(df[['VALOR'] + ['Fecha']].values, df['future'].values):
      sequences.append(entry)
      if len(sequences) == N_STEPS:
          sequence_data.append([np.array(sequences), target])

  last_sequence = list([s[:len(['VALOR'])] for s in sequences]) + list(last_sequence)
  last_sequence = np.array(last_sequence).astype(np.float32)

  # construct the X's and Y's
  X, Y = [], []
  for seq, target in sequence_data:
      X.append(seq)
      Y.append(target)

  # convert to numpy arrays
  X = np.array(X)
  Y = np.array(Y)

  return df, last_sequence, X, Y

In [102]:
#Entrenando el modelo
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['VALOR']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [103]:
#MUCHO CUIDADO, Se demora aproximadamente 2 horas y media predecir el precio para 3 día, y solo se realizan 3 loops
#Predicciones
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['VALOR'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
1454/1454 [==============================] - 39s 22ms/step - loss: 0.0014
Epoch 2/80
1454/1454 [==============================] - 38s 26ms/step - loss: 5.6084e-04
Epoch 3/80
1454/1454 [==============================] - 38s 26ms/step - loss: 5.4617e-04
Epoch 4/80
1454/1454 [==============================] - 37s 25ms/step - loss: 5.5030e-04
Epoch 5/80
1454/1454 [==============================] - 35s 24ms/step - loss: 5.3808e-04
Epoch 6/80
1454/1454 [==============================] - 36s 25ms/step - loss: 5.2481e-04
Epoch 7/80
1454/1454 [==============================] - 33s 23ms/step - loss: 5.1764e-04
Epoch 8/80
1454/1454 [==============================] - 32s 22ms/step - loss: 5.0579e-04
Epoch 9/80
1454/1454 [==============================] - 31s 21ms/step - loss: 4.7542e-04
Epoch 10/80
1454/1454 [==============================] - 40s 27ms/step - loss: 4.6487e-04
Epoch 11/80
1454/1454 [==============================] - 33s 23ms/step - loss: 4.6498e-04
Epoch 12/80
1454/1454 [

In [105]:
#Vemos la predicción
if bool(predictions) == True and len(predictions) >0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message=f'La predicción para los siguientes 3 días son({predictions_str})'

print(message)
#Nos da $4338.89, $4236.19, $4298.77

La predicción para los siguientes 3 días son(4338.89$, 4236.19$, 4298.77$)


In [ ]:
# Grafica para el modelo (Dado que la predicción se realizo usando los datos del 13/10/2023 existe un error de compatibilidad en la grafica de la predicción)
copy_df = Serie.copy()
y_predicted = model.predict(x_train)
y_predicted_transformed = np.squeeze(scaler.inverse_transform(y_predicted))
first_seq = scaler.inverse_transform(np.expand_dims(y_train[:6], axis=1))
last_seq = scaler.inverse_transform(np.expand_dims(y_train[-3:], axis=1))
y_predicted_transformed = np.append(first_seq, y_predicted_transformed)
y_predicted_transformed = np.append(y_predicted_transformed, last_seq)
copy_df[f'predicted_close'] = y_predicted_transformed

# Add predicted results to the table
date_now = dt.date.today()-timedelta(days=1)
date_tomorrow = dt.date.today() + dt.timedelta(days=0)
date_after_tomorrow = dt.date.today() + dt.timedelta(days=1)

copy_df.loc[date_now] = [predictions[0], f'{date_now}', 0]
copy_df.loc[date_tomorrow] = [predictions[1], f'{date_tomorrow}', 0]
copy_df.loc[date_after_tomorrow] = [predictions[2], f'{date_after_tomorrow}', 0]

# Result chart
plt.style.use(style='ggplot')
plt.figure(figsize=(16,10))
plt.plot(copy_df['VALOR'][-150:].head(147))
plt.plot(copy_df['predicted_close'][-150:].head(147), linewidth=1, linestyle='dashed')
plt.plot(copy_df['VALOR'][-150:].tail(4))
plt.xlabel('days')
plt.ylabel('price')
plt.legend([f'Precio Actual Dolar',
            f'Precio Predicho',
            f'Predicción para los 3 dias'])
plt.show()

In [127]:
#Ahora analisis con serie de tiempo

#Importamos los Librerias para hacer el analisis de series temporales
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

In [156]:
Serie1 = new_df.sort_values(by='Fecha', ascending=True)

In [ ]:
Serie1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

# Leer tu DataFrame "Serie" (asegúrate de que la columna "fecha" esté en formato datetime)
# Supongamos que ya tienes tu DataFrame "Serie"
# serie = pd.read_csv('tu_archivo.csv')
# serie['fecha'] = pd.to_datetime(serie['fecha'])

# Establecer la columna "fecha" como índice
Serie1.set_index('Fecha', inplace=True)

# Verificar la estacionariedad de la serie
result = adfuller(Serie1['VALOR'])
if result[1] <= 0.05:
    print("La serie es estacionaria.")
else:
    print("La serie no es estacionaria. Realizando diferenciación...")
    serie_diff = Serie1.diff().dropna()
    result = adfuller(serie_diff['VALOR'])
    if result[1] <= 0.05:
        print("La serie diferenciada es estacionaria.")
    else:
        print("La serie sigue sin ser estacionaria. Puede ser necesario realizar más diferenciación.")

# Descomponer la serie para identificar componentes estacionales, tendencia y residuos
decomposition = seasonal_decompose(serie_diff, model='aditive')  # Usar descomposición aditiva
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Realizar el análisis ARIMA en la serie de residuos
order = (2, 1, 1)  # Ejemplo de orden del modelo ARIMA (p, d, q)
model = ARIMA(residual, order=order)
model_fit = model.fit()

# Realizar una predicción de los residuos
forecast = model_fit.forecast(steps=10)

# Reconstruir la serie original con la predicción
predicted_series = pd.Series(data=forecast, index=serie_diff.index[-1]+pd.to_timedelta(np.arange(1, 11), unit='D'))
predicted_series = predicted_series + seasonal[-10:]  # Sumar la predicción a la componente estacional

# Graficar la serie original y la predicción
plt.figure(figsize=(12, 6))
plt.plot(Serie1, label='Serie Original')
plt.plot(predicted_series, label='Predicción')
plt.legend(loc='best')